In [10]:
import pandas as pd
import numpy as np
import warnings
from time import time

# --- Módulos de Scikit-learn ---
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

# --- Módulos propios ---
# Importamos la función de normalización del archivo normalization_functions
from normalization_functions import pipeline_a_normalize

# --- Configuración ---
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 150)

print("¡Librerías cargadas!")

¡Librerías cargadas!


In [11]:
# --- Carga de Datos ---
# Usamos los archivos que generaste de la Fase 0

# Ruta relativa desde 'src/' a 'data/processed/train.csv'
TRAIN_PATH = "../data/processed/train.csv"

# Cargamos solo los datos de ENTRENAMIENTO
try:
    df_train = pd.read_csv(TRAIN_PATH)
    df_train.dropna(subset=['text', 'Polarity'], inplace=True) # Asegurar que no hay nulos
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {TRAIN_PATH}")
    print("Asegúrate de haber ejecutado '0_split_data.py' primero.")

# Definimos X_train y y_train
# ¡Nunca tocamos el archivo test.csv en esta fase!
X_train = df_train['text']
y_train = df_train['Polarity']

print(f"Datos de entrenamiento cargados: {len(X_train)} opiniones.")
X_train.head()

Datos de entrenamiento cargados: 24169 opiniones.


0    El mejor también en el lockdown También ahora, abiertos solo con comida para llevar y a domicilio es el mejór de lo mejór: mismo sabor, comida cal...
1    Vacaciones con mi esposo Desde que llegamos al hotel el personal a estado muy atento con nosotros por lo cual estamos muy contentos y con ganas de...
2    Super Bowl Excelente servicio y ponen el súper tazón en todas partes, muy recomendable. La comida es deliciosa, el servicio es excelente, el perso...
3    Rico y abundante Excelente los desayunos muy abundantes gran variedad y combos precios muy accesibles y las malteadas bastante buenas en cuanto al...
4    Tratamiento de spa Tuvimos un masaje de 80 minutos por fin y parejas y Katy, manicura y pedicura por Lupita y Jane. Todos eran excelentes en lo qu...
Name: text, dtype: object

In [ ]:
# --- Experimento 1: Pipeline A + Binario + Naive Bayes ---

print("Iniciando Experimento 1: (Pipeline A + Binario + Naive Bayes)")
start_time = time()

# 1. Definir el pipeline
pipeline_nb_bin = Pipeline([
    # Paso 1: Vectorizador (Representación)
    # Usamos CountVectorizer con binary=True para la representación Binaria
    # También aplicamos la normalización (tu función) directamente aquí
    ('vectorizer', CountVectorizer(
        preprocessor=pipeline_a_normalize,  # <-- Tu función de limpieza
        binary=True                         # <-- Representación Binaria
    )),
    
    # Paso 2: Modelo (Clasificador)
    ('classifier', MultinomialNB())
])

# 2. Definir la estrategia de validación cruzada
# Dividimos el set de entrenamiento en 5 folds (partes) [cite: 33]
kfold = KFold(n_splits=5, shuffle=True, random_state=42) # Usamos 42 para reproducibilidad en la validación

# 3. Definir la métrica de evaluación
# Queremos calcular el F1-Macro [cite: 37, 43]
f1_macro_scorer = make_scorer(f1_score, average='macro')

# 4. Ejecutar la validación cruzada
# Esto entrena y evalúa el pipeline 5 veces, una por cada fold
cv_results = cross_validate(
    pipeline_nb_bin,
    X_train,
    y_train,
    cv=kfold,
    scoring={'f1_macro': f1_macro_scorer}
)

# 5. Calcular y mostrar resultados
end_time = time()
avg_f1 = cv_results['test_f1_macro'].mean()
std_f1 = cv_results['test_f1_macro'].std()
exec_time = end_time - start_time

print("\n--- Resultados (Experimento 1) ---")
print(f"F1-Macro Promedio (5-folds): {avg_f1:.4f} (+/- {std_f1:.4f})")
print(f"Tiempo de ejecución: {exec_time:.2f} segundos")

# Guardamos este resultado para el reporte final
results_log = [{
    'pipeline': 'A',
    'vectorizer': 'Binario',
    'model': 'Naive Bayes',
    'avg_f1_macro': avg_f1,
    'std_f1_macro': std_f1
}]

print("\n¡Experimento 1 completado!")

Iniciando Experimento 1: (Pipeline A + Binario + Naive Bayes)

--- Resultados (Experimento 1) ---
F1-Macro Promedio (5-folds): 0.2701 (+/- 0.0071)
Tiempo de ejecución: 11.02 segundos

¡Experimento 1 completado!


In [13]:
# --- Experimento 2: Pipeline A + Binario + Logistic Regression ---

print("Iniciando Experimento 2: (Pipeline A + Binario + Logistic Regression)")
start_time = time()

# 1. Definir el pipeline
pipeline_lr_bin = Pipeline([
    # Paso 1: Vectorizador (Binario)
    ('vectorizer', CountVectorizer(
        preprocessor=pipeline_a_normalize,
        binary=True
    )),
    
    # Paso 2: Modelo (Clasificador)
    # ¡Cambiamos el clasificador!
    ('classifier', LogisticRegression(
        random_state=42,
        max_iter=1000  # Aumentamos max_iter para asegurar que converja
    ))
])

# 2. Definir la estrategia de validación cruzada (la misma de antes)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Definir la métrica (la misma de antes)
f1_macro_scorer = make_scorer(f1_score, average='macro')

# 4. Ejecutar la validación cruzada
cv_results = cross_validate(
    pipeline_lr_bin,
    X_train,
    y_train,
    cv=kfold,
    scoring={'f1_macro': f1_macro_scorer}
)

# 5. Calcular y mostrar resultados
end_time = time()
avg_f1 = cv_results['test_f1_macro'].mean()
std_f1 = cv_results['test_f1_macro'].std()
exec_time = end_time - start_time

print("\n--- Resultados (Experimento 2) ---")
print(f"F1-Macro Promedio (5-folds): {avg_f1:.4f} (+/- {std_f1:.4f})")
print(f"Tiempo de ejecución: {exec_time:.2f} segundos")

# Guardamos este resultado
results_log.append({
    'pipeline': 'A',
    'vectorizer': 'Binario',
    'model': 'Logistic Regression',
    'avg_f1_macro': avg_f1,
    'std_f1_macro': std_f1
})

print("\n¡Experimento 2 completado!")

Iniciando Experimento 2: (Pipeline A + Binario + Logistic Regression)

--- Resultados (Experimento 2) ---
F1-Macro Promedio (5-folds): 0.4556 (+/- 0.0122)
Tiempo de ejecución: 45.71 segundos

¡Experimento 2 completado!


In [14]:
# --- Experimento 3: Pipeline A + Frecuencia + Naive Bayes ---

print("Iniciando Experimento 3: (Pipeline A + Frecuencia + Naive Bayes)")
start_time = time()

# 1. Definir el pipeline
pipeline_nb_freq = Pipeline([
    # Paso 1: Vectorizador (Representación)
    # Es el mismo CountVectorizer, PERO sin 'binary=True'
    ('vectorizer', CountVectorizer(
        preprocessor=pipeline_a_normalize  # Tu función de limpieza
    )),
    
    # Paso 2: Modelo (Clasificador)
    ('classifier', MultinomialNB())
])

# 2. Definir la estrategia de validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Definir la métrica
f1_macro_scorer = make_scorer(f1_score, average='macro')

# 4. Ejecutar la validación cruzada
cv_results = cross_validate(
    pipeline_nb_freq,
    X_train,
    y_train,
    cv=kfold,
    scoring={'f1_macro': f1_macro_scorer}
)

# 5. Calcular y mostrar resultados
end_time = time()
avg_f1 = cv_results['test_f1_macro'].mean()
std_f1 = cv_results['test_f1_macro'].std()
exec_time = end_time - start_time

print("\n--- Resultados (Experimento 3) ---")
print(f"F1-Macro Promedio (5-folds): {avg_f1:.4f} (+/- {std_f1:.4f})")
print(f"Tiempo de ejecución: {exec_time:.2f} segundos")

# Guardamos este resultado
results_log.append({
    'pipeline': 'A',
    'vectorizer': 'Frecuencia',
    'model': 'Naive Bayes',
    'avg_f1_macro': avg_f1,
    'std_f1_macro': std_f1
})

print("\n¡Experimento 3 completado!")

Iniciando Experimento 3: (Pipeline A + Frecuencia + Naive Bayes)

--- Resultados (Experimento 3) ---
F1-Macro Promedio (5-folds): 0.3059 (+/- 0.0087)
Tiempo de ejecución: 9.70 segundos

¡Experimento 3 completado!


In [15]:
# --- Experimento 4: Pipeline A + Frecuencia + Logistic Regression ---

print("Iniciando Experimento 4: (Pipeline A + Frecuencia + Logistic Regression)")
start_time = time()

# 1. Definir el pipeline
pipeline_lr_freq = Pipeline([
    # Paso 1: Vectorizador (Frecuencia)
    ('vectorizer', CountVectorizer(
        preprocessor=pipeline_a_normalize
    )),
    
    # Paso 2: Modelo (Clasificador)
    ('classifier', LogisticRegression(
        random_state=42,
        max_iter=1000
    ))
])

# 2. Definir la estrategia de validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Definir la métrica
f1_macro_scorer = make_scorer(f1_score, average='macro')

# 4. Ejecutar la validación cruzada
cv_results = cross_validate(
    pipeline_lr_freq,
    X_train,
    y_train,
    cv=kfold,
    scoring={'f1_macro': f1_macro_scorer}
)

# 5. Calcular y mostrar resultados
end_time = time()
avg_f1 = cv_results['test_f1_macro'].mean()
std_f1 = cv_results['test_f1_macro'].std()
exec_time = end_time - start_time

print("\n--- Resultados (Experimento 4) ---")
print(f"F1-Macro Promedio (5-folds): {avg_f1:.4f} (+/- {std_f1:.4f})")
print(f"Tiempo de ejecución: {exec_time:.2f} segundos")

# Guardamos este resultado
results_log.append({
    'pipeline': 'A',
    'vectorizer': 'Frecuencia',
    'model': 'Logistic Regression',
    'avg_f1_macro': avg_f1,
    'std_f1_macro': std_f1
})

print("\n¡Experimento 4 completado!")

Iniciando Experimento 4: (Pipeline A + Frecuencia + Logistic Regression)

--- Resultados (Experimento 4) ---
F1-Macro Promedio (5-folds): 0.4552 (+/- 0.0134)
Tiempo de ejecución: 149.69 segundos

¡Experimento 4 completado!


In [16]:
# --- Experimento 5: Pipeline A + TF-IDF + Naive Bayes ---

print("Iniciando Experimento 5: (Pipeline A + TF-IDF + Naive Bayes)")
start_time = time()

# 1. Definir el pipeline
pipeline_nb_tfidf = Pipeline([
    # Paso 1: Vectorizador (Representación)
    # ¡Cambiamos a TfidfVectorizer!
    ('vectorizer', TfidfVectorizer(
        preprocessor=pipeline_a_normalize  # Tu función de limpieza
    )),
    
    # Paso 2: Modelo (Clasificador)
    ('classifier', MultinomialNB())
])

# 2. Definir la estrategia de validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Definir la métrica
f1_macro_scorer = make_scorer(f1_score, average='macro')

# 4. Ejecutar la validación cruzada
cv_results = cross_validate(
    pipeline_nb_tfidf,
    X_train,
    y_train,
    cv=kfold,
    scoring={'f1_macro': f1_macro_scorer}
)

# 5. Calcular y mostrar resultados
end_time = time()
avg_f1 = cv_results['test_f1_macro'].mean()
std_f1 = cv_results['test_f1_macro'].std()
exec_time = end_time - start_time

print("\n--- Resultados (Experimento 5) ---")
print(f"F1-Macro Promedio (5-folds): {avg_f1:.4f} (+/- {std_f1:.4f})")
print(f"Tiempo de ejecución: {exec_time:.2f} segundos")

# Guardamos este resultado
results_log.append({
    'pipeline': 'A',
    'vectorizer': 'TF-IDF',
    'model': 'Naive Bayes',
    'avg_f1_macro': avg_f1,
    'std_f1_macro': std_f1
})

print("\n¡Experimento 5 completado!")

Iniciando Experimento 5: (Pipeline A + TF-IDF + Naive Bayes)

--- Resultados (Experimento 5) ---
F1-Macro Promedio (5-folds): 0.1636 (+/- 0.0010)
Tiempo de ejecución: 9.52 segundos

¡Experimento 5 completado!


In [17]:
# --- Experimento 6: Pipeline A + TF-IDF + Logistic Regression ---

print("Iniciando Experimento 6: (Pipeline A + TF-IDF + Logistic Regression)")
start_time = time()

# 1. Definir el pipeline
pipeline_lr_tfidf = Pipeline([
    # Paso 1: Vectorizador (TF-IDF)
    ('vectorizer', TfidfVectorizer(
        preprocessor=pipeline_a_normalize
    )),
    
    # Paso 2: Modelo (Clasificador)
    # ¡Cambiamos al clasificador LogisticRegression!
    ('classifier', LogisticRegression(
        random_state=42,
        max_iter=1000
    ))
])

# 2. Definir la estrategia de validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Definir la métrica
f1_macro_scorer = make_scorer(f1_score, average='macro')

# 4. Ejecutar la validación cruzada
cv_results = cross_validate(
    pipeline_lr_tfidf,
    X_train,
    y_train,
    cv=kfold,
    scoring={'f1_macro': f1_macro_scorer}
)

# 5. Calcular y mostrar resultados
end_time = time()
avg_f1 = cv_results['test_f1_macro'].mean()
std_f1 = cv_results['test_f1_macro'].std()
exec_time = end_time - start_time

print("\n--- Resultados (Experimento 6) ---")
print(f"F1-Macro Promedio (5-folds): {avg_f1:.4f} (+/- {std_f1:.4f})")
print(f"Tiempo de ejecución: {exec_time:.2f} segundos")

# Guardamos este último resultado
results_log.append({
    'pipeline': 'A',
    'vectorizer': 'TF-IDF',
    'model': 'Logistic Regression',
    'avg_f1_macro': avg_f1,
    'std_f1_macro': std_f1
})

print("\n¡Experimento 6 completado!")

Iniciando Experimento 6: (Pipeline A + TF-IDF + Logistic Regression)

--- Resultados (Experimento 6) ---
F1-Macro Promedio (5-folds): 0.4025 (+/- 0.0034)
Tiempo de ejecución: 25.17 segundos

¡Experimento 6 completado!


In [19]:
# --- Fin de la Fase 1: Guardar Resultados ---

print("--- Resumen de Resultados (Pipeline A) ---")

# 1. Convertir el log de resultados a un DataFrame
df_results = pd.DataFrame(results_log)
df_results.sort_values(by='avg_f1_macro', ascending=False, inplace=True)

# 2. Definir la ruta de guardado
# Guardaremos esto en la carpeta 'results/'
RESULTS_PATH = "../results/fase_1_pipeline_A.csv"

# 3. Guardar el DataFrame en un archivo CSV
try:
    df_results.to_csv(RESULTS_PATH, index=False, encoding='utf-8')
    print(f"\n¡Éxito! Resultados guardados en: {RESULTS_PATH}")
except Exception as e:
    print(f"Error al guardar los resultados: {e}")

# 4. Mostrar el DataFrame final en el notebook
print("\nResultados finales:")
display(df_results)

--- Resumen de Resultados (Pipeline A) ---

¡Éxito! Resultados guardados en: ../results/fase_1_pipeline_A.csv

Resultados finales:


,pipeline,vectorizer,model,avg_f1_macro,std_f1_macro
1,A,Binario,Logistic Regression,0.455560,0.012169
3,A,Frecuencia,Logistic Regression,0.455174,0.013371
5,A,TF-IDF,Logistic Regression,0.402485,0.003411
2,A,Frecuencia,Naive Bayes,0.305907,0.008735
0,A,Binario,Naive Bayes,0.270102,0.007121
4,A,TF-IDF,Naive Bayes,0.163595,0.001005
